In [1]:
import pandas as pd

import category_encoders as ce
import lightgbm as lgb

from sklearn import metrics

In [2]:
data = pd.read_csv('kickstarter.csv', parse_dates=['launched'])

In [3]:
data = data.query("state != 'live'")
data.shape

(375862, 15)

In [4]:
data = data.assign(outcome=(data['state'] == 'successful').astype(int))
data.head(2)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0


In [5]:
data = data.assign(hour=data.launched.dt.hour, 
                  day=data.launched.dt.day, 
                  month=data.launched.dt.month, 
                  year=data.launched.dt.year)
data.head(2)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0,12,11,8,2015
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0,4,2,9,2017


In [6]:
cat_features = ['category', 'currency', 'country']

In [7]:
X = data[['hour', 'day', 'month', 'year', 'outcome', 'category', 'currency', 'country']]
X.head(2)

,hour,day,month,year,outcome,category,currency,country
0,12,11,8,2015,0,Poetry,GBP,GB
1,4,2,9,2017,0,Narrative Film,USD,US


In [8]:
def get_splits(df, valid_fraction=0.1):
    valid_rows = int(len(df)*valid_fraction)
    train = df[:-valid_rows * 2]
    valid = df[-valid_rows*2: -valid_rows]
    test = df[-valid_rows:]
    
    return train, valid, test

In [9]:
train, valid, test = get_splits(X)
train.head(2)

,hour,day,month,year,outcome,category,currency,country
0,12,11,8,2015,0,Poetry,GBP,GB
1,4,2,9,2017,0,Narrative Film,USD,US


In [10]:
encoder = ce.TargetEncoder(cols=cat_features)
encoder.fit(train[cat_features], train['outcome'])

TargetEncoder(cols=['category', 'currency', 'country'], drop_invariant=False,
              handle_missing='value', handle_unknown='value',
              min_samples_leaf=1, return_df=True, smoothing=1.0, verbose=0)

In [11]:
train_sub = train.drop(cat_features, axis=1)
valid_sub = valid.drop(cat_features, axis=1)
test_sub = test.drop(cat_features, axis=1)

test_sub.head(2)

,hour,day,month,year,outcome
340783,14,27,10,2014,1
340784,18,19,8,2015,0


In [12]:
TE_train = train_sub.join(encoder.transform(train[cat_features]))
TE_valid = valid_sub.join(encoder.transform(valid[cat_features]))
TE_test = test_sub.join(encoder.transform(test[cat_features]))

TE_train.head()

,hour,day,month,year,outcome,category,currency,country
0,12,11,8,2015,0,0.360190,0.357122,0.361636
1,4,2,9,2017,0,0.384615,0.373392,0.376631
2,0,12,1,2013,0,0.384615,0.373392,0.376631
3,3,17,3,2012,0,0.412655,0.373392,0.376631
4,8,4,7,2015,0,0.302625,0.373392,0.376631


In [13]:
def train_model(train, valid, test, feature_cols=None):
    if feature_cols is None:
        feature_cols = train.columns.drop(['outcome'])
        
    dtrain = lgb.Dataset(train[feature_cols], train['outcome'])
    dvalid = lgb.Dataset(valid[feature_cols], valid['outcome'])
    
    param = {'num_leaves': 55, 'objective': 'binary', 'metric': 'auc', 'seed': 7}
    num_rounds = 1000
    
    model = lgb.train(param, dtrain, num_rounds, valid_sets=[dvalid], early_stopping_rounds=10, 
                     verbose_eval=False)
    valid_pred = model.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['outcome'], valid_pred)
    
    print(f"Validation AUC Score: {valid_score}")
    
    test_pred = model.predict(test[feature_cols])
    test_score = metrics.roc_auc_score(test['outcome'], test_pred)
    
    print(f"Test AUC Score: {test_score}")
    return model, valid_score

In [14]:
train_model(TE_train, TE_valid, TE_test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010993 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 276
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
Validation AUC Score: 0.7047351683720786
Test AUC Score: 0.7056059101699441


(<lightgbm.basic.Booster at 0x1ecb3ba9c08>, 0.7047351683720786)